
# 🎓 RNN Training 101

You'll train a simple sequence model (RNN / LSTM / GRU) on a synthetic time series and **visualize loss curves and predictions**.

**Learning goals**
- Understand what **sequence data** is and how to build time-series samples with a sliding window: shape **(batch, seq_len, features)**.
- Implement and compare **RNN / LSTM / GRU** forward passes and training loops.
- Plot the **training loss** and **model predictions vs. ground truth**.
- Experiment with hyperparameters and observe their effects.

> How to use: Run each cell from top to bottom. Works out-of-the-box in Google Colab.


In [ ]:

# ✅ Environment & Reproducibility
import math, random, os, sys, time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from typing import Tuple, List, Optional

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

print('PyTorch:', torch.__version__)
print('Device:', device)


PyTorch: 2.8.0+cu126
Device: cuda



## 1) What is **sequence data**?

Sequence data is any data where **order matters** and each element may depend on previous elements. Common examples:
- **Time series** (sensor readings, stock prices, weather).  
- **Language** (characters/words in a sentence).  
- **Audio** (waveform samples over time).  

Key concepts you should know:
- **Causality:** to predict the next point at time `t+1`, the model can only use information up to time `t`.
- **Windowing:** we make training samples by taking a **fixed-length window** of past values (length = `seq_len`) as input `X`, and a **target** as either the **next** value (one-step ahead) or a **future segment** (multi-step).
- **Shapes:** we usually arrange a mini-batch as `(batch, seq_len, features)`. For a univariate time series, `features=1`.
- **Overlapping vs. non-overlapping windows:** overlapping gives more samples and smoother learning; non-overlapping reduces redundancy.
- **Normalization:** standardize/scale values to stabilize training.
- **Train/Validation split:** split along the **time axis** to avoid leaking future information into the past.
- **Padding/Masking (advanced):** for variable-length sequences, we may pad to the same length and use masks.




## 2) Build a simple time-series dataset (sine mix + noise)

**Task:** given the past `seq_len` points, **predict the next point** (one-step ahead regression).  
We create a long series by mixing several sine waves, add light noise, then cut it into many `(X, y)` samples using a **sliding window**.

**Data pipeline overview**
1. Generate a long normalized series `series[t]`.
2. Split into **train** (earlier times) and **validation** (later times) to respect causality.
3. Use a sliding window of length `seq_len` to build input `X` and target `y`:
   - `X[i] = series[i : i+seq_len]`
   - `y[i] = series[i+seq_len]` (the next value)
4. Stack windows into tensors with shapes:
   - `X`: `(N, seq_len, 1)`
   - `y`: `(N, 1)`



### 🔍 Example: Creating sequence samples step by step

Let's start from a **simple numeric sequence**:


$$
\text{series} = [1, 2, 3, 4, 5, 6, 7, 8]
$$

If we choose `seq_len = 3`,  
each training input `X` is the last 3 numbers, and the target `y` is the **next** number.

| Sample Index | Input Sequence `X` | Target `y` |
| :----------- | :----------------- | :--------- |
| 0            | [1, 2, 3]          | 4          |
| 1            | [2, 3, 4]          | 5          |
| 2            | [3, 4, 5]          | 6          |
| 3            | [4, 5, 6]          | 7          |
| 4            | [5, 6, 7]          | 8          |

So our dataset becomes:
- **X shape:** (5 samples, 3 time steps, 1 feature)
- **y shape:** (5 samples, 1 value)

This is exactly how we prepare inputs for RNNs — each row in `X` is a small **time window** used to predict the next step.


In [ ]:
# 2A) Dataset class
class SineMixDataset(Dataset):
    def __init__(self, total_len=6000, seq_len=30, train=True, train_ratio=0.8):
        super().__init__()
        self.seq_len = seq_len
        t = np.linspace(0, 200, total_len).astype(np.float32)
        # Mix multiple sine waves with different frequencies/phases + mild noise
        x = (np.sin(0.3*t) + 0.5*np.sin(0.05*t + 1.0) + 0.3*np.sin(0.9*t + 0.5)).astype(np.float32)
        x += 0.05 * np.random.randn(total_len).astype(np.float32)
        # Normalize
        x = (x - x.mean()) / (x.std() + 1e-6)
        # Train/Val split along time
        split = int(total_len * train_ratio)
        if train:
            self.series = x[:split]
        else:
            self.series = x[split:]
        # Build (X, y) windows using a sliding window
        X, y = [], []
        for i in range(0, len(self.series)-seq_len-1):
            X.append(self.series[i:i+seq_len])
            y.append(self.series[i+seq_len])
        self.X = np.array(X, dtype=np.float32)[:, :, None]   # (N, seq_len, 1)
        self.y = np.array(y, dtype=np.float32)[:, None]      # (N, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.y[idx])


# 2B) Quick sanity check

The purpose of the “Quick sanity check” is to verify that the dataset is generated correctly, with proper shapes and data structure, and that it can be loaded by the DataLoader without errors.

In [ ]:
train_ds = SineMixDataset(train=True, seq_len=30)
val_ds   = SineMixDataset(train=False, seq_len=30)

print("Train samples:", len(train_ds), "| Val samples:", len(val_ds))
x0, y0 = train_ds[0]
print("One sample X shape:", x0.shape, "y shape:", y0.shape)  # (seq_len, 1), (1,)

batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)

for xb, yb in train_loader:
    print('Batch X shape:', xb.shape, '| Batch y shape:', yb.shape)
    break


Train samples: 4769 | Val samples: 1169
One sample X shape: torch.Size([30, 1]) y shape: torch.Size([1])
Batch X shape: torch.Size([64, 30, 1]) | Batch y shape: torch.Size([64, 1])


### 2D) Inspect a few samples from the dataset


In [ ]:
# Pick one training sample and print its input and target
sample_X, sample_y = train_ds[0]
print("Sample 0 input (X) shape:", sample_X.shape)
print("Sample 0 target (y) shape:", sample_y.shape)
#print("First few values of X:", sample_X.squeeze()[:31].numpy())
#print("Target y value:", float(sample_y))

# Optional: visualize a portion of the training sequence and highlight first window
plt.figure(figsize=(8,3))
plt.plot(train_ds.series[:500], label='Training series')
plt.axvspan(0, train_ds.seq_len, color='orange', alpha=0.2, label='Example input window')
plt.axvline(train_ds.seq_len, color='red', linestyle='--', label='Target position')
plt.title("Example: Sliding window on sine-mix data")
plt.xlabel("Time step")
plt.ylabel("Normalized value")
plt.legend()
plt.show()

### 2E) Visualize one full sequence and its target point

In [ ]:

sample_index = 0
sample_X, sample_y = train_ds[sample_index]

# Print all 30 input values and the target
print(f"Sample {sample_index} input (X) shape: {sample_X.shape}")
print(f"Sample {sample_index} target (y) shape: {sample_y.shape}\n")

print("Full input sequence (30 steps):")
print(np.round(sample_X.squeeze().numpy(), 4))  # Rounded for readability
print(f"\nTarget y (next value): {float(sample_y):.4f}")

# Visualize
plt.figure(figsize=(8,3))
plt.plot(range(len(sample_X)), sample_X.squeeze().numpy(), 'b-o', label='Input sequence (X)')
plt.plot(len(sample_X), float(sample_y), 'ro', label='Target (next step y)')
plt.title("One complete sample: 30-step input and 1-step target")
plt.xlabel("Time step within the sample")
plt.ylabel("Signal value")
plt.legend()
plt.show()


## 3) Define a flexible RNN model (RNN / LSTM / GRU)

### 🕒 Step-by-Step Sequence Processing in LSTM

Each sequence has shape **(B, T, F)**

* **B** — batch size (how many sequences are processed together)
* **T** — number of time steps
* **F** — feature dimension of each time step

During training, we go **through time step by step**:

1. At **t = 1**, the network sees the first input `x[:, 0, :]`
   → It uses the initial memory **(h₀, c₀)** (both zeros).
2. The LSTM cell outputs new states **(h₁, c₁)**.
3. At **t = 2**, it takes the next input `x[:, 1, :]`
   together with **(h₁, c₁)**, producing **(h₂, c₂)**.
4. This repeats until **t = T**, where we obtain the final hidden state **h_T**.

The key idea 💡:

> The hidden state `h` and cell state `c` act as the **memory** of the sequence,
> carrying information from the past to influence future predictions.

---

**In code:**

```python
# Go through each time step
for t in range(T):
    h, c = self.cell(x[:, t, :], h, c)
```

This loop means:

* Each `x[:, t, :]` is one moment in time.
* `h` and `c` are continuously updated.
* The final `h` (after all time steps) summarizes the whole sequence.



In [ ]:

class ManualLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        # Each gate has its own weight for input (W) and hidden (U)
        self.W_i = nn.Linear(input_size, hidden_size)
        self.U_i = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_f = nn.Linear(input_size, hidden_size)
        self.U_f = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_o = nn.Linear(input_size, hidden_size)
        self.U_o = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_c = nn.Linear(input_size, hidden_size)
        self.U_c = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, x_t, h_prev, c_prev):
        # --- Compute gates ---
        i_t = torch.sigmoid(self.W_i(x_t) + self.U_i(h_prev))   # input gate
        f_t = torch.sigmoid(self.W_f(x_t) + self.U_f(h_prev))   # forget gate
        o_t = torch.sigmoid(self.W_o(x_t) + self.U_o(h_prev))   # output gate
        g_t = torch.tanh(self.W_c(x_t) + self.U_c(h_prev))      # candidate cell

        # --- Update states ---
        c_t = f_t * c_prev + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        return h_t, c_t


class SeqRegressor(nn.Module):
    """
    Flexible sequence-to-value regression model supporting RNN/LSTM/GRU architectures.
    
    This model processes sequential input data and predicts a single continuous value.
    It uses the final hidden state from the recurrent layers as the sequence representation.
    
    Architecture:
        Input → RNN/LSTM/GRU Layer(s) → Dropout → Linear → Output
    
    Args:
        input_size (int): Number of input features at each time step (default: 1 for univariate)
        hidden_size (int): Dimension of hidden state in recurrent layers (default: 64)
        num_layers (int): Number of stacked recurrent layers (default: 1)
        cell_type (str): Type of recurrent cell - 'rnn', 'lstm', or 'gru' (default: 'lstm')
        dropout (float): Dropout probability between recurrent layers (default: 0.2)
                        Only applied when num_layers > 1
        bidirectional (bool): Whether to use bidirectional recurrent layers (default: False)
    
    Shape:
        Input: (batch_size, seq_len, input_size)
        Output: (batch_size, 1)
    
    Example:
        >>> model = SeqRegressor(input_size=1, hidden_size=128, num_layers=2, cell_type='lstm')
        >>> x = torch.randn(32, 30, 1)  # batch=32, seq_len=30, features=1
        >>> y = model(x)  # Output shape: (32, 1)
    """
    
    def __init__(
        self, 
        input_size: int = 1,
        hidden_size: int = 64,
        num_layers: int = 1,
        cell_type: str = 'lstm',
        dropout: float = 0.2,
        bidirectional: bool = False
    ):
        super(SeqRegressor, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type.lower()
        self.bidirectional = bidirectional
        
        # Validate cell type
        valid_cells = {'rnn', 'lstm', 'gru'}
        if self.cell_type not in valid_cells:
            raise ValueError(f"cell_type must be one of {valid_cells}, got '{self.cell_type}'")
        
        # Select appropriate recurrent layer
        if self.cell_type == 'rnn':
            self.rnn = nn.RNN(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout if num_layers > 1 else 0,
                bidirectional=bidirectional
            )
        elif self.cell_type == 'lstm':
            self.rnn = nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout if num_layers > 1 else 0,
                bidirectional=bidirectional
            )
        else:  # gru
            self.rnn = nn.GRU(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout if num_layers > 1 else 0,
                bidirectional=bidirectional
            )
        
        # Additional dropout after recurrent layers
        self.dropout = nn.Dropout(dropout)
        
        # Output layer
        # If bidirectional, the final hidden size is doubled
        fc_input_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc = nn.Linear(fc_input_size, 1)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        """
        Initialize weights using Xavier/Glorot initialization for better gradient flow.
        This helps prevent vanishing/exploding gradients in deep recurrent networks.
        """
        for name, param in self.named_parameters():
            if 'weight_ih' in name:
                # Input-to-hidden weights
                nn.init.xavier_uniform_(param)
            elif 'weight_hh' in name:
                # Hidden-to-hidden weights (use orthogonal for RNNs)
                nn.init.orthogonal_(param)
            elif 'bias' in name:
                # Initialize biases to zero
                nn.init.zeros_(param)
                # For LSTM, initialize forget gate bias to 1 (helps learning)
                if self.cell_type == 'lstm':
                    n = param.size(0)
                    param.data[n//4:n//2].fill_(1.0)
            elif 'fc' in name and 'weight' in name:
                # Final linear layer
                nn.init.xavier_uniform_(param)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the sequence regressor.
        
        Args:
            x: Input tensor of shape (batch_size, seq_len, input_size)
        
        Returns:
            Output predictions of shape (batch_size, 1)
        
        Process:
            1. Pass input through recurrent layers
            2. Extract final hidden state
            3. Apply dropout
            4. Project to output dimension
        """
        # x shape: (batch, seq_len, input_size)
        
        # Pass through recurrent layers
        # output shape: (batch, seq_len, hidden_size * num_directions)
        # h_n shape: (num_layers * num_directions, batch, hidden_size)
        if self.cell_type == 'lstm':
            output, (h_n, c_n) = self.rnn(x)
        else:  # rnn or gru
            output, h_n = self.rnn(x)
        
        # Extract final hidden state
        # For bidirectional models, concatenate forward and backward final states
        if self.bidirectional:
            # h_n shape: (num_layers * 2, batch, hidden_size)
            # Take last layer's forward and backward states
            h_forward = h_n[-2, :, :]  # Forward direction of last layer
            h_backward = h_n[-1, :, :]  # Backward direction of last layer
            h_final = torch.cat([h_forward, h_backward], dim=1)
        else:
            # Take the last layer's hidden state
            h_final = h_n[-1, :, :]  # Shape: (batch, hidden_size)
        
        # Apply dropout
        h_final = self.dropout(h_final)
        
        # Project to output
        out = self.fc(h_final)  # Shape: (batch, 1)
        
        return out
    
    def get_model_info(self) -> dict:
        """
        Get comprehensive model information for logging and debugging.
        
        Returns:
            Dictionary containing model architecture details and parameter counts
        """
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
        return {
            'architecture': self.__class__.__name__,
            'cell_type': self.cell_type.upper(),
            'input_size': self.input_size,
            'hidden_size': self.hidden_size,
            'num_layers': self.num_layers,
            'bidirectional': self.bidirectional,
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'parameters_K': total_params / 1e3,
            'parameters_M': total_params / 1e6
        }


# ============================================================================
# Instantiate and Test Model
# ============================================================================

device = "cuda" if torch.cuda.is_available() else "cpu"

# Create LSTM model with default configuration
model = SeqRegressor(
    input_size=1, 
    hidden_size=64, 
    num_layers=1, 
    cell_type='lstm',
    dropout=0.2,
    bidirectional=False
).to(device)

# Display model information
model_info = model.get_model_info()
print("=" * 70)
print("MODEL ARCHITECTURE")
print("=" * 70)
print(f"Architecture:         {model_info['architecture']}")
print(f"Cell Type:            {model_info['cell_type']}")
print(f"Input Size:           {model_info['input_size']}")
print(f"Hidden Size:          {model_info['hidden_size']}")
print(f"Number of Layers:     {model_info['num_layers']}")
print(f"Bidirectional:        {model_info['bidirectional']}")
print(f"Total Parameters:     {model_info['total_parameters']:,}")
print(f"Trainable Parameters: {model_info['trainable_parameters']:,}")
print(f"Model Size:           {model_info['parameters_K']:.2f}K parameters")
print("=" * 70)
print("\nModel Structure:")
print("-" * 70)
print(model)
print("=" * 70)

# Test forward pass
test_input = torch.randn(4, 30, 1).to(device)  # (batch=4, seq_len=30, features=1)
test_output = model(test_input)
print(f"\nTest Forward Pass:")
print(f"Input shape:  {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print(f"Output range: [{test_output.min().item():.4f}, {test_output.max().item():.4f}]")
print("=" * 70)



## 4) Train!

We'll train with MSE loss and Adam. After training, we'll plot the loss curve and validation predictions.

**This section is split into two cells** to keep code readable:
- **4A** defines the training function.
- **4B** runs training and draws the loss curves.





---

### 🧩 1. The difference between **time steps (seq_len)** and **layers (num_layers)**

| Concept                   | Meaning                                                                                                | Example                                                                  |
| ------------------------- | ------------------------------------------------------------------------------------------------------ | ------------------------------------------------------------------------ |
| **Time step (`seq_len`)** | The number of steps in the input sequence — how many moments the model processes sequentially.         | `seq_len = 30` means each input sequence has 30 consecutive data points. |
| **Layer (`num_layers`)**  | The number of stacked RNN/LSTM layers — how many layers of hidden units are applied at each time step. | `num_layers = 1` means only one LSTM layer is used at each time step.    |

---

### ⚙️ 2. How a single LSTM layer processes 30 time steps

When you feed an input of shape `(batch=64, seq_len=30, features=1)` into the model,
PyTorch’s LSTM automatically iterates **30 times** over the time dimension.

The logic is as follows:

1️⃣ **Step 1:** Input the first time point (x_1) with the initial hidden and cell states (h_0, c_0).
→ Output new states (h_1, c_1).

2️⃣ **Step 2:** Input the next time point (x_2), together with (h_1, c_1).
→ Output (h_2, c_2).

3️⃣ Continue this process until **step 30**, producing the final hidden state (h_{30}),
which summarizes the entire input sequence.

---

### 🧱 3. What “one layer” really means

Setting `num_layers = 1` means that at each time step, the LSTM cell has only **one layer of computation**.
That is:

* Along the **time dimension**, the network still performs **30 sequential updates**.
* Along the **depth (vertical stacking)** dimension, there is only one layer — no additional LSTMs stacked above it.

---

### ✅ In one sentence

> Even with only one LSTM layer, the network still processes all 30 time steps in sequence.
> **“num_layers controls depth, seq_len controls length.”**
> A single-layer LSTM can read the entire 30-step sequence and update its memory at each step.


### 🧠 How a single-layer LSTM processes 30 time steps

```
       ┌────────────────────────────────────────────────────────────┐
       │                    One training batch                      │
       │        shape = (batch_size=64, seq_len=30, features=1)     │
       └────────────────────────────────────────────────────────────┘
                            │
                            ▼
            ┌────────────────────────────────┐
            │        Single LSTM Layer       │
            └────────────────────────────────┘
                            │
                            ▼
 ┌─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
 │  x₁         │  x₂         │  x₃         │ ...         │  x₃₀        │
 │ (t=1)       │ (t=2)       │ (t=3)       │             │ (t=30)      │
 ├─────────────┴─────────────┴─────────────┴─────────────┴─────────────┤
 │  h₀,c₀ ──▶ LSTM ──▶ h₁,c₁ ──▶ LSTM ──▶ h₂,c₂ ──▶ ... ──▶ h₃₀,c₃₀ │
 └──────────────────────────────────────────────────────────────────────┘
                            │
                            ▼
              Take the last hidden state (h₃₀)
                            │
                            ▼
                 Linear layer → Predicted value (ŷ)


```


In [ ]:
# 4A) Define training function

def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    epochs: int = 15,
    lr: float = 1e-3,
    grad_clip: Optional[float] = None,
    patience: int = 10,
    verbose: bool = True
) -> Tuple[List[float], List[float]]:
    """
    Train a sequence model with comprehensive logging and early stopping.
    
    Args:
        model: PyTorch model to train
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        epochs: Maximum number of training epochs
        lr: Learning rate for Adam optimizer
        grad_clip: Maximum gradient norm (None to disable clipping)
        patience: Number of epochs to wait for improvement before early stopping
        verbose: Whether to print detailed training progress
    
    Returns:
        Tuple of (train_losses, val_losses) lists
    """
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    
    train_losses, val_losses = [], []
    best_val_loss = float('inf')
    best_epoch = 0
    best_state = None
    epochs_without_improvement = 0
    
    # Training configuration display
    if verbose:
        print("=" * 70)
        print("TRAINING CONFIGURATION")
        print("=" * 70)
        print(f"Optimizer:            Adam")
        print(f"Learning Rate:        {lr}")
        print(f"Loss Function:        MSE")
        print(f"Gradient Clipping:    {grad_clip if grad_clip else 'Disabled'}")
        print(f"Early Stopping:       Enabled (patience={patience})")
        print(f"Max Epochs:           {epochs}")
        print(f"Train Batches:        {len(train_loader)}")
        print(f"Val Batches:          {len(val_loader)}")
        print("=" * 70)
        print(f"\n{'Epoch':<8}{'Train Loss':<15}{'Val Loss':<15}{'Best':<8}{'Time(s)':<10}{'Status'}")
        print("-" * 70)
    
    total_start_time = time.time()
    
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        
        # ========== Training Phase ==========
        model.train()
        train_running_loss = 0.0
        train_samples = 0
        
        for batch_idx, (xb, yb) in enumerate(train_loader):
            xb, yb = xb.to(device), yb.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            predictions = model(xb)
            loss = loss_fn(predictions, yb)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping (prevents exploding gradients)
            if grad_clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            
            optimizer.step()
            
            train_running_loss += loss.item() * xb.size(0)
            train_samples += xb.size(0)
        
        train_loss = train_running_loss / train_samples
        train_losses.append(train_loss)
        
        # ========== Validation Phase ==========
        model.eval()
        val_running_loss = 0.0
        val_samples = 0
        
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                predictions = model(xb)
                loss = loss_fn(predictions, yb)
                val_running_loss += loss.item() * xb.size(0)
                val_samples += xb.size(0)
        
        val_loss = val_running_loss / val_samples
        val_losses.append(val_loss)
        
        epoch_time = time.time() - epoch_start_time
        
        # ========== Model Checkpoint & Early Stopping ==========
        is_best = val_loss < best_val_loss
        status = ""
        
        if is_best:
            best_val_loss = val_loss
            best_epoch = epoch
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            epochs_without_improvement = 0
            status = "NEW BEST"
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                status = "EARLY STOP"
        
        # Display progress
        if verbose:
            best_marker = "*" if is_best else ""
            print(f"{epoch:<8}{train_loss:<15.6f}{val_loss:<15.6f}{best_marker:<8}{epoch_time:<10.2f}{status}")
        
        # Early stopping check
        if epochs_without_improvement >= patience:
            if verbose:
                print("-" * 70)
                print(f"Early stopping triggered after {epoch} epochs")
                print(f"Best validation loss: {best_val_loss:.6f} at epoch {best_epoch}")
            break
    
    total_time = time.time() - total_start_time
    
    # Load best model
    if best_state is not None:
        model.load_state_dict(best_state)
        if verbose:
            print("-" * 70)
            print(f"Loaded best model from epoch {best_epoch}")
    
    # Training summary
    if verbose:
        print("=" * 70)
        print("TRAINING SUMMARY")
        print("=" * 70)
        print(f"Total Training Time:  {total_time:.2f}s")
        print(f"Epochs Completed:     {epoch}/{epochs}")
        print(f"Best Epoch:           {best_epoch}")
        print(f"Best Val Loss:        {best_val_loss:.6f}")
        print(f"Final Train Loss:     {train_losses[-1]:.6f}")
        print(f"Final Val Loss:       {val_losses[-1]:.6f}")
        print(f"Improvement:          {(train_losses[0] - train_losses[-1]) / train_losses[0] * 100:.2f}%")
        print("=" * 70)
    
    # ========== Visualization ==========
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    
    # Loss curves
    ax1 = axes[0]
    epochs_range = range(1, len(train_losses) + 1)
    ax1.plot(epochs_range, train_losses, 'b-', label='Training Loss', linewidth=2)
    ax1.plot(epochs_range, val_losses, 'r-', label='Validation Loss', linewidth=2)
    ax1.axvline(x=best_epoch, color='g', linestyle='--', alpha=0.7, label=f'Best Epoch ({best_epoch})')
    ax1.set_xlabel('Epoch', fontsize=11)
    ax1.set_ylabel('MSE Loss', fontsize=11)
    ax1.set_title('Training and Validation Loss Curves', fontsize=12, fontweight='bold')
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # Loss reduction visualization
    ax2 = axes[1]
    train_reduction = [(train_losses[0] - loss) / train_losses[0] * 100 for loss in train_losses]
    val_reduction = [(val_losses[0] - loss) / val_losses[0] * 100 for loss in val_losses]
    ax2.plot(epochs_range, train_reduction, 'b-', label='Train Loss Reduction', linewidth=2)
    ax2.plot(epochs_range, val_reduction, 'r-', label='Val Loss Reduction', linewidth=2)
    ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax2.set_xlabel('Epoch', fontsize=11)
    ax2.set_ylabel('Loss Reduction (%)', fontsize=11)
    ax2.set_title('Learning Progress (% Improvement)', fontsize=12, fontweight='bold')
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return train_losses, val_losses
    plt.ylabel('MSE Loss')
    plt.title('Training & Validation Loss')
    plt.legend()
    plt.show()

    return train_losses, val_losses


In [ ]:

# 4B) Run training
_ = train_model(model, train_loader, val_loader, epochs=15, lr=1e-3, grad_clip=1.0)



## 5) Inspect predictions vs. ground truth

We take a few validation sequences and compare predicted next-step values with the true targets.


In [ ]:

"""
Comprehensive Prediction Analysis
==================================
Evaluate model performance on validation data with detailed visualizations
and statistical metrics.
"""

# ========== Generate Predictions ==========
model.eval()
xb, yb = next(iter(val_loader))

with torch.no_grad():
    predictions = model(xb.to(device)).cpu().numpy().squeeze()

ground_truth = yb.numpy().squeeze()

# Calculate metrics
mse = mean_squared_error(ground_truth, predictions)
mae = mean_absolute_error(ground_truth, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(ground_truth, predictions)
correlation = np.corrcoef(ground_truth, predictions)[0, 1]

# Calculate residuals
residuals = ground_truth - predictions
mean_residual = np.mean(residuals)
std_residual = np.std(residuals)

# ========== Display Metrics ==========
print("=" * 70)
print("PREDICTION PERFORMANCE METRICS")
print("=" * 70)
print(f"Mean Squared Error (MSE):     {mse:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"Mean Absolute Error (MAE):    {mae:.6f}")
print(f"R² Score:                     {r2:.6f}")
print(f"Correlation Coefficient:      {correlation:.6f}")
print("-" * 70)
print(f"Residual Mean:                {mean_residual:.6f}")
print(f"Residual Std Dev:             {std_residual:.6f}")
print(f"Max Error:                    {np.max(np.abs(residuals)):.6f}")
print("=" * 70)

# ========== Visualizations ==========
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Predictions vs Ground Truth (Time Series)
ax1 = fig.add_subplot(gs[0, :])
sample_indices = np.arange(len(ground_truth))
ax1.plot(sample_indices, ground_truth, 'b-', label='Ground Truth', linewidth=2, alpha=0.7)
ax1.plot(sample_indices, predictions, 'r--', label='Predictions', linewidth=2, alpha=0.7)
ax1.fill_between(sample_indices, ground_truth, predictions, alpha=0.2, color='gray', label='Error')
ax1.set_xlabel('Sample Index in Batch', fontsize=11)
ax1.set_ylabel('Normalized Value', fontsize=11)
ax1.set_title('Next-Step Predictions vs Ground Truth', fontsize=12, fontweight='bold')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# 2. Scatter Plot: Predictions vs Ground Truth
ax2 = fig.add_subplot(gs[1, 0])
ax2.scatter(ground_truth, predictions, alpha=0.5, s=30, edgecolors='k', linewidth=0.5)
min_val = min(ground_truth.min(), predictions.min())
max_val = max(ground_truth.max(), predictions.max())
ax2.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
ax2.set_xlabel('Ground Truth', fontsize=11)
ax2.set_ylabel('Predictions', fontsize=11)
ax2.set_title(f'Scatter Plot (R² = {r2:.4f})', fontsize=12, fontweight='bold')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)
ax2.axis('equal')

# 3. Residual Plot
ax3 = fig.add_subplot(gs[1, 1])
ax3.scatter(predictions, residuals, alpha=0.5, s=30, edgecolors='k', linewidth=0.5)
ax3.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Zero Residual')
ax3.axhline(y=mean_residual, color='g', linestyle=':', linewidth=2, label=f'Mean ({mean_residual:.4f})')
ax3.fill_between([predictions.min(), predictions.max()], 
                  -2*std_residual, 2*std_residual, 
                  alpha=0.2, color='yellow', label='±2σ')
ax3.set_xlabel('Predicted Values', fontsize=11)
ax3.set_ylabel('Residuals (Truth - Pred)', fontsize=11)
ax3.set_title('Residual Analysis', fontsize=12, fontweight='bold')
ax3.legend(loc='best')
ax3.grid(True, alpha=0.3)

# 4. Residual Distribution (Histogram)
ax4 = fig.add_subplot(gs[2, 0])
ax4.hist(residuals, bins=30, edgecolor='black', alpha=0.7, color='skyblue')
ax4.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Zero')
ax4.axvline(x=mean_residual, color='g', linestyle=':', linewidth=2, label=f'Mean ({mean_residual:.4f})')
ax4.set_xlabel('Residual Value', fontsize=11)
ax4.set_ylabel('Frequency', fontsize=11)
ax4.set_title('Residual Distribution', fontsize=12, fontweight='bold')
ax4.legend(loc='best')
ax4.grid(True, alpha=0.3, axis='y')

# 5. Error Distribution (Absolute Error)
ax5 = fig.add_subplot(gs[2, 1])
abs_errors = np.abs(residuals)
ax5.hist(abs_errors, bins=30, edgecolor='black', alpha=0.7, color='salmon')
ax5.axvline(x=mae, color='b', linestyle='--', linewidth=2, label=f'MAE ({mae:.4f})')
ax5.set_xlabel('Absolute Error', fontsize=11)
ax5.set_ylabel('Frequency', fontsize=11)
ax5.set_title('Absolute Error Distribution', fontsize=12, fontweight='bold')
ax5.legend(loc='best')
ax5.grid(True, alpha=0.3, axis='y')

plt.suptitle('Comprehensive Prediction Analysis on Validation Batch', 
             fontsize=14, fontweight='bold', y=0.995)
plt.show()

# ========== Sample-by-Sample Comparison (First 10) ==========
print("\nSample-by-Sample Comparison (First 10 samples):")
print("-" * 70)
print(f"{'Index':<8}{'Ground Truth':<18}{'Prediction':<18}{'Error':<15}")
print("-" * 70)
for i in range(min(10, len(ground_truth))):
    error = ground_truth[i] - predictions[i]
    print(f"{i:<8}{ground_truth[i]:<18.6f}{predictions[i]:<18.6f}{error:<15.6f}")
print("-" * 70)



## 6) Comprehensive Exercises and Experiments

### Exercise 1: Architecture Comparison (RNN vs LSTM vs GRU)

**Objective:** Compare the performance of different recurrent architectures on the same task.

**Tasks:**
1. Train models with RNN, LSTM, and GRU cells (same hyperparameters)
2. Compare final validation losses
3. Analyze which architecture learns fastest
4. Examine prediction quality differences

**Expected Insights:**
- LSTM/GRU typically outperform vanilla RNN on long sequences
- LSTM has more parameters but better gradient flow
- GRU is a good middle ground (fewer parameters, similar performance)


In [ ]:
"""
Exercise 1: RNN vs LSTM vs GRU Comparison
"""

# Configuration
architectures = ['rnn', 'lstm', 'gru']
results = {}

print("=" * 70)
print("EXERCISE 1: ARCHITECTURE COMPARISON")
print("=" * 70)

for arch in architectures:
    print(f"\n{'='*70}")
    print(f"Training {arch.upper()} Model")
    print(f"{'='*70}")
    
    # Create model
    model_arch = SeqRegressor(
        input_size=1,
        hidden_size=64,
        num_layers=1,
        cell_type=arch,
        dropout=0.2
    ).to(device)
    
    # Display model info
    info = model_arch.get_model_info()
    print(f"Parameters: {info['total_parameters']:,} ({info['parameters_K']:.2f}K)")
    
    # Train
    train_losses, val_losses = train_model(
        model_arch,
        train_loader,
        val_loader,
        epochs=20,
        lr=1e-3,
        grad_clip=1.0,
        patience=7,
        verbose=True
    )
    
    # Store results
    results[arch] = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'final_train': train_losses[-1],
        'final_val': val_losses[-1],
        'best_val': min(val_losses),
        'params': info['total_parameters']
    }

# ========== Comparison Visualization ==========
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training loss comparison
ax1 = axes[0]
for arch in architectures:
    ax1.plot(results[arch]['train_losses'], label=f'{arch.upper()}', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=11)
ax1.set_ylabel('Training Loss', fontsize=11)
ax1.set_title('Training Loss Comparison', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Validation loss comparison
ax2 = axes[1]
for arch in architectures:
    ax2.plot(results[arch]['val_losses'], label=f'{arch.upper()}', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=11)
ax2.set_ylabel('Validation Loss', fontsize=11)
ax2.set_title('Validation Loss Comparison', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ========== Results Summary ==========
print("\n" + "=" * 70)
print("ARCHITECTURE COMPARISON RESULTS")
print("=" * 70)
print(f"{'Architecture':<15}{'Parameters':<15}{'Best Val Loss':<18}{'Final Val Loss':<18}")
print("-" * 70)
for arch in architectures:
    r = results[arch]
    print(f"{arch.upper():<15}{r['params']:<15,}{r['best_val']:<18.6f}{r['final_val']:<18.6f}")
print("=" * 70)

# Find best architecture
best_arch = min(results.items(), key=lambda x: x[1]['best_val'])
print(f"\nBest Architecture: {best_arch[0].upper()} (Val Loss: {best_arch[1]['best_val']:.6f})")
print("=" * 70)



### Exercise 2: Hidden Size Impact

**Objective:** Understand how hidden layer size affects model capacity and performance.

**Tasks:**
1. Train models with hidden sizes: [32, 64, 128, 256]
2. Track parameter count vs performance
3. Identify the optimal hidden size
4. Check for underfitting vs overfitting

**Expected Insights:**
- Larger hidden sizes = more capacity but more parameters
- Too small → underfitting, too large → overfitting/slower training
- Optimal size depends on data complexity


In [ ]:
"""
Exercise 2: Hidden Size Experiment
"""

hidden_sizes = [32, 64, 128, 256]
hidden_results = {}

print("=" * 70)
print("EXERCISE 2: HIDDEN SIZE IMPACT")
print("=" * 70)

for h_size in hidden_sizes:
    print(f"\n{'='*70}")
    print(f"Training with Hidden Size = {h_size}")
    print(f"{'='*70}")
    
    model_h = SeqRegressor(
        input_size=1,
        hidden_size=h_size,
        num_layers=1,
        cell_type='lstm',
        dropout=0.2
    ).to(device)
    
    info = model_h.get_model_info()
    print(f"Parameters: {info['total_parameters']:,} ({info['parameters_K']:.2f}K)")
    
    train_losses, val_losses = train_model(
        model_h,
        train_loader,
        val_loader,
        epochs=20,
        lr=1e-3,
        grad_clip=1.0,
        patience=7,
        verbose=True
    )
    
    hidden_results[h_size] = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'params': info['total_parameters'],
        'best_val': min(val_losses),
        'final_val': val_losses[-1],
        'overfitting_gap': train_losses[-1] - val_losses[-1]
    }

# ========== Visualization ==========
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss curves
ax1 = axes[0, 0]
for h_size in hidden_sizes:
    ax1.plot(hidden_results[h_size]['val_losses'], label=f'H={h_size}', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Validation Loss vs Hidden Size', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Parameters vs Performance
ax2 = axes[0, 1]
params = [hidden_results[h]['params'] for h in hidden_sizes]
best_vals = [hidden_results[h]['best_val'] for h in hidden_sizes]
ax2.plot(params, best_vals, 'bo-', linewidth=2, markersize=8)
for h in hidden_sizes:
    ax2.annotate(f'H={h}', (hidden_results[h]['params'], hidden_results[h]['best_val']),
                textcoords="offset points", xytext=(0,10), ha='center')
ax2.set_xlabel('Number of Parameters')
ax2.set_ylabel('Best Validation Loss')
ax2.set_title('Model Capacity vs Performance', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Overfitting analysis
ax3 = axes[1, 0]
gaps = [hidden_results[h]['overfitting_gap'] for h in hidden_sizes]
ax3.bar([str(h) for h in hidden_sizes], gaps, color='coral', edgecolor='black')
ax3.set_xlabel('Hidden Size')
ax3.set_ylabel('Train-Val Gap')
ax3.set_title('Overfitting Analysis (Train - Val Loss)', fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')

# Summary table visualization
ax4 = axes[1, 1]
ax4.axis('off')
table_data = []
for h in hidden_sizes:
    r = hidden_results[h]
    table_data.append([
        f'{h}',
        f'{r["params"]:,}',
        f'{r["best_val"]:.6f}',
        f'{r["overfitting_gap"]:.6f}'
    ])

table = ax4.table(cellText=table_data,
                 colLabels=['Hidden\nSize', 'Parameters', 'Best Val\nLoss', 'Overfit\nGap'],
                 cellLoc='center',
                 loc='center',
                 colWidths=[0.15, 0.25, 0.25, 0.25])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
ax4.set_title('Performance Summary', fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

# Best hidden size
best_h = min(hidden_results.items(), key=lambda x: x[1]['best_val'])
print(f"\nOptimal Hidden Size: {best_h[0]} (Val Loss: {best_h[1]['best_val']:.6f})")
print("=" * 70)



### Exercise 3: Sequence Length Impact

**Objective:** Investigate how input sequence length affects prediction accuracy.

**Tasks:**
1. Train models with different sequence lengths: [10, 20, 30, 50]
2. Analyze how longer context improves/hurts predictions
3. Examine training efficiency vs sequence length

**Expected Insights:**
- Longer sequences provide more context but slower training
- Very short sequences may lack sufficient information
- Optimal length depends on temporal dependencies in data


In [ ]:
"""
Exercise 3: Sequence Length Experiment
"""

seq_lengths = [10, 20, 30, 50]
seq_results = {}

print("=" * 70)
print("EXERCISE 3: SEQUENCE LENGTH IMPACT")
print("=" * 70)

for seq_len in seq_lengths:
    print(f"\n{'='*70}")
    print(f"Training with Sequence Length = {seq_len}")
    print(f"{'='*70}")
    
    # Create datasets with specific sequence length
    train_ds_seq = SineMixDataset(train=True, seq_len=seq_len)
    val_ds_seq = SineMixDataset(train=False, seq_len=seq_len)
    
    train_loader_seq = DataLoader(train_ds_seq, batch_size=64, shuffle=True, drop_last=True)
    val_loader_seq = DataLoader(val_ds_seq, batch_size=64, shuffle=False, drop_last=False)
    
    print(f"Train samples: {len(train_ds_seq)}, Val samples: {len(val_ds_seq)}")
    
    # Create and train model
    model_seq = SeqRegressor(
        input_size=1,
        hidden_size=64,
        num_layers=1,
        cell_type='lstm',
        dropout=0.2
    ).to(device)
    
    train_losses, val_losses = train_model(
        model_seq,
        train_loader_seq,
        val_loader_seq,
        epochs=20,
        lr=1e-3,
        grad_clip=1.0,
        patience=7,
        verbose=True
    )
    
    seq_results[seq_len] = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'best_val': min(val_losses),
        'final_val': val_losses[-1],
        'n_samples': len(train_ds_seq)
    }

# ========== Visualization ==========
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss comparison
ax1 = axes[0]
for seq_len in seq_lengths:
    ax1.plot(seq_results[seq_len]['val_losses'], label=f'SeqLen={seq_len}', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Validation Loss')
ax1.set_title('Validation Loss vs Sequence Length', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Best performance vs sequence length
ax2 = axes[1]
best_vals = [seq_results[sl]['best_val'] for sl in seq_lengths]
ax2.plot(seq_lengths, best_vals, 'ro-', linewidth=2, markersize=10)
for sl in seq_lengths:
    ax2.annotate(f'{seq_results[sl]["best_val"]:.5f}',
                (sl, seq_results[sl]['best_val']),
                textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)
ax2.set_xlabel('Sequence Length')
ax2.set_ylabel('Best Validation Loss')
ax2.set_title('Optimal Sequence Length Analysis', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
print("\n" + "=" * 70)
print("SEQUENCE LENGTH COMPARISON")
print("=" * 70)
print(f"{'Seq Length':<15}{'Train Samples':<18}{'Best Val Loss':<20}{'Final Val Loss'}")
print("-" * 70)
for sl in seq_lengths:
    r = seq_results[sl]
    print(f"{sl:<15}{r['n_samples']:<18}{r['best_val']:<20.6f}{r['final_val']:.6f}")
print("=" * 70)

best_seq = min(seq_results.items(), key=lambda x: x[1]['best_val'])
print(f"\nOptimal Sequence Length: {best_seq[0]} (Val Loss: {best_seq[1]['best_val']:.6f})")
print("=" * 70)



## 7) Conclusion and Key Takeaways

### What You've Learned

1. **Sequence Data Fundamentals**
   - How to structure time series data for RNN training
   - Importance of sliding windows and proper train/val splits
   - Shape conventions: (batch, seq_len, features)

2. **Recurrent Architectures**
   - Implemented and compared RNN, LSTM, and GRU
   - Understanding of hidden states and temporal dependencies
   - How recurrent layers process sequences step-by-step

3. **Training Best Practices**
   - Gradient clipping prevents exploding gradients
   - Early stopping prevents overfitting
   - Proper validation monitoring is crucial

4. **Hyperparameter Impact**
   - Architecture choice (LSTM usually best for long dependencies)
   - Hidden size balances capacity vs efficiency
   - Sequence length affects both performance and computation

### Professional Development Skills Gained

- Type hints and comprehensive docstrings
- Structured logging and progress tracking
- Professional visualization techniques
- Modular, reusable code design
- Comprehensive error analysis

### Next Steps

- Apply to real-world time series (stock prices, weather, sensor data)
- Explore attention mechanisms and Transformers
- Try bidirectional models for non-causal tasks
- Implement multi-step ahead prediction
- Add more sophisticated regularization techniques
